In [ ]:
import sys
import time

import dcelmesh
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse

from linear_geodesic_optimization.mesh.rectangle import Mesh as RectangleMesh
from linear_geodesic_optimization.optimization.geodesic import Computer as Geodesic
from linear_geodesic_optimization.optimization.laplacian import Computer as Laplacian
from linear_geodesic_optimization.plot import get_heat_map

In [ ]:
seed = time.time_ns()
seed = seed % (2**32 - 1)
print(f'Seed: {seed}')
rng = np.random.default_rng(seed)

In [ ]:
width = 30
height = 30

mesh = RectangleMesh(width, height)
topology = mesh.get_topology()
laplacian = Laplacian(mesh)
source_index = mesh.nearest_vertex(np.array([0., 0.])).index()
target_indices = [
    mesh.nearest_vertex(np.array([0.25, 0.25])).index(),
    mesh.nearest_vertex(np.array([0.25, -0.25])).index(),
]
geodesic = Geodesic(mesh, source_index, target_indices, laplacian, 1000.)

z = mesh.set_parameters(rng.random(width * height))
dz = rng.random(width * height)
# dz = np.zeros(topology.n_vertices())
# dz[mesh.nearest_vertex(np.array([0., 0.])).index()] = 1.
dz = 1e-7 * dz / np.linalg.norm(dz)

In [ ]:
def f(geodesic: Geodesic, z):
    mesh.set_parameters(z)
    geodesic.forward()
    return geodesic.distances[target_indices[0]]

def g(geodesic: Geodesic, z, dz):
    mesh.set_parameters(z)
    geodesic.reverse()
    return geodesic.dif_distances[target_indices[0]] @ dz / np.linalg.norm(dz)

In [ ]:
quantity_z = np.array(f(geodesic, z)).flatten()
dif_quantity = np.array(g(geodesic, z, dz)).flatten()
quantity_z_dz = np.array(f(geodesic, z + dz)).flatten()
estimated_dif_quantity = (quantity_z_dz - quantity_z) / np.linalg.norm(dz)

In [ ]:
quotient = np.linalg.norm(dif_quantity) / np.linalg.norm(estimated_dif_quantity)
print(f'Quotient of magnitudes: {quotient:.6f}')

angle = np.arccos(dif_quantity @ estimated_dif_quantity
                  / (np.linalg.norm(dif_quantity)
                     * np.linalg.norm(estimated_dif_quantity)))
print(f'Angle between:          {angle:.6f}')